# Training MaskRCNN using TFOD

In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from object_detection.utils import label_map_util
import PIL.Image
import io

In [2]:
label_map = label_map_util.load_labelmap('labelmap.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)

In [3]:
model_name = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz'
model_url = 'download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz'

In [10]:
!curl -L {model_url} > {model_name}; tar -xvf {model_name}; rm {model_name};

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  462M  100  462M    0     0  7819k      0  0:01:00  0:01:00 --:--:-- 8416k62M   67  311M    0     0  7671k      0  0:01:01  0:00:41  0:00:20 8524k
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/checkpoint
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/saved_model.pb
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/assets/
mask_rcnn_inception_resnet_v2_1024x1024_c

In [4]:
model_dir = r'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8'

In [25]:
with open('mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config', 'w') as f:
    f.write(r'''# Mask R-CNN with Inception Resnet v2 (no atrous)
# Sync-trained on COCO (with 8 GPUs) with batch size 16 (1024x1024 resolution)
# Initialized from Imagenet classification checkpoint
#
# Train on GPU-8
#
# Achieves 40.4 box mAP and 35.5 mask mAP on COCO17 val

model {
  faster_rcnn {
    number_of_stages: 3
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 500
        width: 500
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_resnet_v2_keras'
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 17
    maxpool_kernel_size: 1
    maxpool_stride: 1
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        mask_height: 33
        mask_width: 33
        mask_prediction_conv_depth: 0
        mask_prediction_num_conv_layers: 4
        conv_hyperparams {
          op: CONV
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.01
            }
          }
        }
        predict_instance_masks: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
    second_stage_mask_prediction_loss_weight: 4.0
    resize_masks: false
  }
}

train_config: {
  batch_size: 16
  num_steps: 200
  freeze_variables: []
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: 0.008
          total_steps: 200
          warmup_learning_rate: 0.0
          warmup_steps: 50
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  label_map_path: "labelmap.pbtxt"
  tf_record_input_reader {
    input_path: "pet_faces_train.tfrecord"
  }
  load_instance_masks: true
  mask_type: PNG_MASKS
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  metrics_set: "coco_mask_metrics"
  eval_instance_masks: true
  use_moving_averages: false
  batch_size: 1
  include_metrics_per_category: true
}

eval_input_reader: {
  label_map_path: "labelmap.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "pet_faces_train.tfrecord"
  }
  load_instance_masks: true
  mask_type: PNG_MASKS
}
''')

In [13]:
!cp ~/my_workspace/tfod-model-garden/research/object_detection/model_main_tf2.py .

In [15]:
!python model_main_tf2.py --pipeline_config_path=mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config --model_dir=mask_rcnn_training --alsologtostderr

Traceback (most recent call last):
  File "model_main_tf2.py", line 31, in <module>
    import tensorflow.compat.v2 as tf
ModuleNotFoundError: No module named 'tensorflow'
